In [9]:


!pip install -q ultralytics timm torch torchvision pandas pillow matplotlib tifffile imagecodecs scikit-learn tqdm

import os, random, numpy as np, pandas as pd, tifffile as tiff
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import torch, timm, matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
EXCEL_PATH = "/content/drive/MyDrive/gmeruli.xlsx"
DATA_DIR   = "/content/drive/MyDrive/Converted"   # Folder with .ome.tif slides
YOLO_DIR   = "/content/drive/MyDrive/Glom_YOLOv8"
os.makedirs(YOLO_DIR, exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("▶ Device:", device)


Mounted at /content/drive
▶ Device: cuda


In [10]:
# Load Excel
df = pd.read_excel(EXCEL_PATH)
df = df.head(1000)
df["label"] = df["Type"].apply(lambda x: 1 if str(x).lower().strip()=="gs" else 0)
df["Glom_Index"] = df["Glom_Index"].fillna(0).astype(int)
df["split"] = df.apply(lambda _: "train" if random.random()<0.8 else "val", axis=1)

# Create YOLO directories
for s in ["train","val"]:
    os.makedirs(f"{YOLO_DIR}/images/{s}", exist_ok=True)
    os.makedirs(f"{YOLO_DIR}/labels/{s}", exist_ok=True)

# Convert Excel annotations → YOLO .txt labels AND copy corresponding TIF images
count_valid = 0
import cv2

for _, row in df.iterrows():
    W,H = row["SVS_image_Width"], row["SVS_image_Height"]
    if pd.isna(W) or pd.isna(H): continue

    # Calculate normalized YOLO coordinates
    x_c,y_c = row["center_x"]/W, row["center_y"]/H
    w_n,h_n = row["Width"]/W, row["Height"]/H
    if not (0 <= x_c <= 1 and 0 <= y_c <= 1): continue

    # Find corresponding TIF file (exact name matching)
    folder_name = row['Folder_Name']
    tif_files = list(Path(DATA_DIR).glob(f"{folder_name}.tif")) + list(Path(DATA_DIR).glob(f"{folder_name}.tiff"))

    if tif_files:
        tif_path = tif_files[0]  # Take the first match

        try:
            # Load and convert TIF to JPG for YOLO
            tif_image = tiff.imread(str(tif_path))

            # Handle different TIF formats
            if len(tif_image.shape) == 3:
                if tif_image.shape[0] < tif_image.shape[2]:  # Channel first to channel last
                    tif_image = np.transpose(tif_image, (1, 2, 0))
                if tif_image.shape[2] > 3:  # Take first 3 channels if more than RGB
                    tif_image = tif_image[:, :, :3]
            else:
                # Convert grayscale to RGB
                tif_image = np.stack([tif_image] * 3, axis=-1)

            # Normalize to 0-255 range
            if tif_image.max() > 255:
                tif_image = ((tif_image - tif_image.min()) / (tif_image.max() - tif_image.min()) * 255).astype(np.uint8)
            else:
                tif_image = tif_image.astype(np.uint8)

            # Save as JPG for YOLO
            img_filename = f"{folder_name}_{row['Glom_Index']}.jpg"
            img_path = f"{YOLO_DIR}/images/{row['split']}/{img_filename}"

            # Convert RGB to BGR for OpenCV
            tif_image_bgr = cv2.cvtColor(tif_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(img_path, tif_image_bgr)

            # Create corresponding label file
            txt_path = f"{YOLO_DIR}/labels/{row['split']}/{folder_name}_{row['Glom_Index']}.txt"
            with open(txt_path, "w") as f:
                f.write(f"{row['label']} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

            count_valid += 1

        except Exception as e:
            print(f"⚠️ Error processing {tif_path}: {e}")
            continue
    else:
        print(f"⚠️ No TIF file found for {folder_name}")

print(f"✅ Generated {count_valid} YOLO image-label pairs.")


⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF file found for 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
⚠️ No TIF 

In [11]:
# Debug: Check available TIF files and Excel data structure
print("🔍 Debugging TIF files and Excel structure...")

# Check TIF files in DATA_DIR
tif_files = list(Path(DATA_DIR).glob("*.tif")) + list(Path(DATA_DIR).glob("*.tiff"))
print(f"📁 Found {len(tif_files)} TIF files in {DATA_DIR}:")
for i, tif_file in enumerate(tif_files[:5]):  # Show first 5
    print(f"   {i+1}. {tif_file.name}")
if len(tif_files) > 5:
    print(f"   ... and {len(tif_files)-5} more files")

# Check Excel data structure
print(f"\n📊 Excel data info:")
print(f"   Rows: {len(df)}")
print(f"   Columns: {list(df.columns)}")
print(f"\n📋 Sample folder names from Excel:")
sample_folders = df['Folder_Name'].unique()[:5]
for folder in sample_folders:
    print(f"   - {folder}")

# Check for missing coordinate columns
required_cols = ['center_x', 'center_y', 'Width', 'Height', 'SVS_image_Width', 'SVS_image_Height']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"⚠️ Missing columns: {missing_cols}")
    print(f"📋 Available columns: {list(df.columns)}")
else:
    print("✅ All required coordinate columns found")

print("\n" + "="*50)

🔍 Debugging TIF files and Excel structure...
📁 Found 20 TIF files in /content/drive/MyDrive/Converted:
   1. 1656e66b-88bb-44b8-84f0-5d1cd23489cb_seg.tif
   2. 06901256-7e47-4184-a792-61f65bc6abbc_seg.tif
   3. 09760768-b3ca-4276-b6f7-1d5ebf2f908e_seg.tif
   4. 16154dc1-b1c2-4ed7-9701-0ee32cf795df_seg.tif
   5. 048f1fb6-6925-4752-a334-d4f0170f16ed_seg.tif
   ... and 15 more files

📊 Excel data info:
   Rows: 1000
   Columns: ['SVSSeqID', 'Folder_Name', 'SVS_image_Width', 'SVS_image_Height', 'Type', 'Glom_Index', 'x1', 'y1', 'x2', 'y2', 'Width', 'Height', 'Area', 'center_x', 'center_y', 'r', 'label', 'split']

📋 Sample folder names from Excel:
   - 20ee2351-a9d6-4f8b-b7b5-8a2e9fba6ce8_seg
   - a444f67d-d737-4b99-acf6-808c08af647b_seg
   - b674fd19-1f87-4b11-96cb-c7df7ab2c791_seg
   - 6ef59e9f-efd5-49bf-9fd1-d392e3095ebb_seg
   - 6d932c19-d74b-438c-bf66-13ce230fa832_seg
✅ All required coordinate columns found



In [12]:
yaml_path = f"{YOLO_DIR}/glom.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""
train: {YOLO_DIR}/images/train
val: {YOLO_DIR}/images/val
nc: 2
names: ['non-GS','GS']
""")
print("✅ Dataset YAML:", yaml_path)


✅ Dataset YAML: /content/drive/MyDrive/Glom_YOLOv8/glom.yaml


In [13]:
# Verify YOLO dataset before training
def verify_yolo_dataset(yolo_dir):
    """Verify that images and labels are properly created"""
    train_imgs = len(list(Path(f"{yolo_dir}/images/train").glob("*.jpg")))
    val_imgs = len(list(Path(f"{yolo_dir}/images/val").glob("*.jpg")))
    train_labels = len(list(Path(f"{yolo_dir}/labels/train").glob("*.txt")))
    val_labels = len(list(Path(f"{yolo_dir}/labels/val").glob("*.txt")))

    print(f"📊 Dataset Verification:")
    print(f"   Training images: {train_imgs}")
    print(f"   Training labels: {train_labels}")
    print(f"   Validation images: {val_imgs}")
    print(f"   Validation labels: {val_labels}")

    if train_imgs == 0:
        print("❌ ERROR: No training images found!")
        return False
    elif train_imgs != train_labels:
        print("⚠️ WARNING: Mismatch between images and labels in training set")

    if val_imgs == 0:
        print("❌ ERROR: No validation images found!")
        return False
    elif val_imgs != val_labels:
        print("⚠️ WARNING: Mismatch between images and labels in validation set")

    print("✅ Dataset verification passed!")
    return True

# Verify dataset
dataset_ok = verify_yolo_dataset(YOLO_DIR)

if not dataset_ok:
    print("❌ Dataset verification failed. Please check the TIF files and folder names.")
else:
    print("🎯 Ready for YOLO training!")

📊 Dataset Verification:
   Training images: 0
   Training labels: 5340
   Validation images: 0
   Validation labels: 3200
❌ ERROR: No training images found!
❌ Dataset verification failed. Please check the TIF files and folder names.


In [15]:
import tifffile as tiff
from PIL import Image
from tqdm import tqdm
import numpy as np

# Directory structure check
for s in ["train", "val"]:
    os.makedirs(f"{YOLO_DIR}/images/{s}", exist_ok=True)
    os.makedirs(f"{YOLO_DIR}/labels/{s}", exist_ok=True)

def crop_patch_from_tif_safe(slide_path, x1, y1, w, h, out_path):
    """Crop region safely from a .tif file, auto-clamping to image bounds."""
    with tiff.TiffFile(str(slide_path)) as tf:
        arr = tf.pages[0].asarray()

    H, W = arr.shape[:2]
    x1, y1 = int(max(0, x1)), int(max(0, y1))
    x2, y2 = int(min(W, x1 + w)), int(min(H, y1 + h))

    # Skip if invalid (zero-size crop)
    if x2 <= x1 or y2 <= y1:
        return False

    crop = arr[y1:y2, x1:x2]
    if crop.size == 0:
        return False

    # Normalize intensity safely
    crop = np.clip(crop, 0, np.percentile(crop, 99.9))
    m = np.max(crop)
    if m == 0 or np.isnan(m):
        return False
    crop = (crop / m * 255).astype(np.uint8)

    Image.fromarray(crop).convert("RGB").save(out_path)
    return True


print("🔧 Generating YOLO training images from Excel (safe mode)...")
count, skipped = 0, 0

for _, row in tqdm(df.iterrows(), total=len(df)):
    folder = row["Folder_Name"]
    glom_idx = int(row["Glom_Index"])
    split = row["split"]

    tif_matches = list(Path(DATA_DIR).glob(f"{folder.replace('_seg','')}.tif"))
    if not tif_matches:
        skipped += 1
        continue

    tif_path = tif_matches[0]
    out_img = f"{YOLO_DIR}/images/{split}/{folder}_{glom_idx}.png"

    ok = crop_patch_from_tif_safe(
        tif_path,
        row["x1"], row["y1"],
        row["Width"], row["Height"],
        out_img
    )
    if ok:
        count += 1
    else:
        skipped += 1

print(f"✅ Saved {count} cropped patches")


🔧 Generating YOLO training images from Excel (safe mode)...


100%|██████████| 1000/1000 [00:00<00:00, 2676.56it/s]

✅ Saved 0 cropped patches
⚠️ Skipped 1000 invalid/missing regions


In [27]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # Start with small model
model.train(data=yaml_path, epochs=50, imgsz=512, val=False)
print("✅ YOLOv8 trained for glomeruli detection.")


Ultralytics 8.3.219 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Glom_YOLOv8/glom.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

In [33]:
import torch, timm, numpy as np
from PIL import Image
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

dino = timm.create_model("vit_small_patch14_dinov2", pretrained=True)
dino.eval().to(device)

@torch.no_grad()
def extract_dino_features(pil_image):
    """Safe universal DINO feature extractor."""
    pil_image = pil_image.resize((518, 518), Image.BICUBIC)
    arr = np.asarray(pil_image, dtype=np.float32) / 255.0
    arr = arr.transpose(2, 0, 1)  # HWC → CHW
    x = torch.tensor(arr[None], dtype=torch.float32, device=device)

    feats = dino.forward_features(x)
    # Handle both tensor or dict outputs
    if isinstance(feats, dict):
        if "x_norm_clstoken" in feats:
            feats = feats["x_norm_clstoken"]
        elif "last_hidden_state" in feats:
            feats = feats["last_hidden_state"][:, 0, :]
        else:
            feats = list(feats.values())[0]
    elif isinstance(feats, torch.Tensor):
        feats = feats[:, 0, :] if feats.ndim == 3 else feats  # CLS token
    else:
        raise ValueError("Unexpected feature output type:", type(feats))

    return feats.cpu().numpy().flatten()

In [34]:
features, labels = [], []
for _, row in tqdm(df.iterrows(), total=len(df)):
    fname = f"{PATCH_DIR}/{row['Folder_Name']}_{int(row['Glom_Index'])}.png"
    if not os.path.exists(fname):
        continue
    try:
        img = Image.open(fname).convert("RGB")
        fvec = extract_dino_features(img)
        features.append(fvec)
        labels.append(row["label"])
    except Exception as e:
        print(f"⚠️ Skipped {fname}: {e}")
        continue

features = np.array(features)
labels = np.array(labels)
print("✅ Feature matrix shape:", features.shape)

100%|██████████| 1000/1000 [00:00<00:00, 5628.42it/s]

✅ Feature matrix shape: (2, 384)


In [38]:
features = np.asarray(features)
labels = np.asarray(labels)

print("Samples:", features.shape, "Labels:", np.bincount(labels))

clf = LogisticRegression(max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


Samples: (2, 384) Labels: [1 1]


In [52]:

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print("\n=== DINOv3 + Classifier Results")
print(classification_report(labels, y_pred, target_names=["non-GS", "GS"]))
print( "\naccuracy score: ",accuracy_score(labels, y_pred))
cm = confusion_matrix(labels, y_pred)
# disp = ConfusionMatrixDisplay(cm, display_labels=["non-GS", "GS"])
# ax = disp.plot(cmap="Blues")
# ax.figure_.suptitle("Confusion Matrix", fontsize=12)
# plt.show()

joblib.dump(clf, "/content/drive/MyDrive/dinov3_logreg.pkl")
print("\n✅ Model saved to /content/drive/MyDrive/dinov3logreg.pkl")


=== DINOv3 + Classifier Results
              precision    recall  f1-score   support

      non-GS       0.50      1.00      0.67         1
          GS       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2


accuracy score:  0.5

precision:  0.0

✅ Model saved to /content/drive/MyDrive/dinov3logreg.pkl


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
